# Importação de pacotes

In [21]:
import pandas as pd
from pycaret.regression import *

# Carregando os dados

In [22]:
# carregando os dados para o formato DataFrame
df = pd.read_csv("insurance_clean.csv", sep=";")

# imprimindo as 5 primeiras linhas
df.head()

,age,sex,bmi,children,smoker,region,charges,age_discretized,bmi_description,children_description,charges_log,smoker_bmi
0,19,female,27.900,0,yes,southwest,16884.92400,0.0,overweight,no children,9.734176,yes_not_obese
1,18,male,33.770,1,no,southeast,1725.55230,0.0,obese,up to 3 children,7.453302,no
2,28,male,33.000,3,no,southeast,4449.46200,1.0,obese,up to 3 children,8.400538,no
3,33,male,22.705,0,no,northwest,21984.47061,1.0,normal,no children,9.998092,no
4,32,male,28.880,0,no,northwest,3866.85520,1.0,overweight,no children,8.260197,no


# Separando dados em treino e teste

A função `sample()` do Pandas DataFrame é usada para obter uma amostra aleatória de linhas ou colunas de um DataFrame. A função retorna um novo DataFrame que contém uma fração aleatória de linhas ou colunas do DataFrame original.

O parâmetro `frac` determina a fração de linhas que você deseja retornar em vez de um número fixo. Neste caso é usado somente 10% do dados para formar o conjunto de teste e 90% para formar os nossos dados de treino.

In [23]:
# separando os dados em teste e treino
test = df.sample(frac=0.10)
train = df.drop(test.index)

# redefinindo o index do DataFrame descartando o indice anterior (drop=True) e realizando a alteração no próprio DataFrame (inplace=True) 
test.reset_index(inplace=True, drop=True)
train.reset_index(inplace=True, drop=True)

In [24]:
# verificando o formato dos conjuntos
print(f"Shape do conjunto de treino {train.shape}")
print(f"Shape do conjunto de teste {test.shape}")

Shape do conjunto de treino (1204, 12)
Shape do conjunto de teste (134, 12)


# Setup

O método `setup()` do PyCaret é usado para preparar os dados para modelagem de aprendizado de máquina. Ele executa várias etapas de pré-processamento de dados, como:

* Separar o conjunto de dados em treinamento e teste.
* Lidar com valores ausentes e valores discrepantes (outliers).
* Codificar variáveis categóricas.
* Normalizar ou padronizar variáveis numéricas.
* Selecionar recursos (features) importantes.
* Balancear os dados.

O método `setup()` retorna um objeto contendo várias informações sobre o pré-processamento de dados que foi realizado, bem como uma pré-visualização dos dados transformados. A partir desse objeto, você pode criar, treinar e avaliar modelos de aprendizado de máquina usando o PyCaret.

In [25]:
# criando o setup do PyCaret
reg = setup(data=train,
            target="charges",
            train_size=0.7,
            preprocess=True,
            normalize=True,
            fold_strategy="kfold",
            fold=10,
            session_id=123)

,Description,Value
0,session_id,123
1,Target,charges
2,Original Data,"(1204, 12)"
3,Missing Values,False
4,Numeric Features,4
5,Categorical Features,7
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(842, 24)"


# Compare models

O método `compare_models` do PyCaret é usado para comparar o desempenho de vários modelos de aprendizado de máquina em um conjunto de dados, usando uma métrica de avaliação específica.

Ao chamar o método `compare_models`, o PyCaret executa automaticamente um experimento de modelagem de aprendizado de máquina usando vários algoritmos de modelagem diferentes. Ele usa um método de validação cruzada para treinar e avaliar cada modelo, dividindo o conjunto de dados em vários subconjuntos e avaliando o desempenho do modelo em cada subconjunto.

O resultado do `compare_models` é uma tabela de classificação de desempenho dos modelos, ordenados pela métrica de avaliação escolhida.

In [26]:
best = compare_models(round=2, sort="MSE")

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,90.87,66373.56,198.92,1.00,0.01,0.01,0.02
dt,Decision Tree Regressor,87.02,77088.73,233.18,1.00,0.01,0.01,0.00
rf,Random Forest Regressor,59.06,118399.68,207.68,1.00,0.01,0.00,0.06
et,Extra Trees Regressor,100.28,394625.82,447.73,1.00,0.02,0.01,0.06
lightgbm,Light Gradient Boosting Machine,156.58,554471.28,551.85,1.00,0.02,0.01,0.01
ada,AdaBoost Regressor,649.94,748758.93,862.11,0.99,0.25,0.17,0.02
llar,Lasso Least Angle Regression,2120.63,8903278.18,2972.20,0.94,0.59,0.35,0.00
lasso,Lasso Regression,2116.52,8916808.70,2974.49,0.94,0.54,0.35,0.24
ridge,Ridge Regression,2123.14,8920246.65,2974.80,0.94,0.55,0.36,0.00
br,Bayesian Ridge,2121.06,8921200.58,2975.09,0.94,0.54,0.35,0.01


# Métricas de regressão

Existem diversas métricas para avaliar a qualidade de modelos de regressão, mas as principais são:

## 1. Erro médio absoluto (MAE)

<center><img src="https://drive.google.com/uc?export=view&id=1kgZEOQ1nuM-rssQ5ElC2pvn1crEUfz1V" width="30%"></p></center>

É a média das diferenças absolutas entre as previsões e os valores reais. É uma métrica robusta e fácil de entender, pois ela mede o tamanho do erro em unidades da variável dependente.

**FÓRMULA:**

$$
\LARGE MAE = \frac{1}{n}\sum_{i=1}^{n}|y_i-\hat{y_i}|
$$


onde:

* $n$ é o número de observações

* $y_i$ é o valor real da i-ésima observação

* $\hat{y_i}$ é o valor previsto pelo modelo para a i-ésima observação

* O símbolo $\left|  \right|$ representa o valor absoluto.

<center><img src="https://drive.google.com/uc?export=view&id=12ThAjcxkGwvqt7JUsbZ4kutHgjiUdrto" width="50%"></p></center>

**VANTAGENS:**

* Fácil interpretação: o MAE é uma métrica simples e fácil de entender, pois representa a média da diferença absoluta entre as previsões e os valores reais.
* Robustez a valores atípicos: o MAE é menos sensível a valores extremos ou outliers do que outras métricas, como o Erro Quadrático Médio (MSE).
* Proporção direta com o erro: o MAE é proporcional ao erro médio do modelo, o que significa que quanto menor o MAE, melhor é o desempenho do modelo.
* Todos os erros individuais são tratados da mesma maneira, o que pode ser uma problema para lidar com outliers.

**DESVANTAGENS:**

* Pode ignorar a direção do erro: o MAE não leva em consideração a direção do erro (ou seja, se o modelo subestima ou superestima a previsão), o que pode ser uma desvantagem em algumas aplicações em que a direção do erro é importante.
* Não é tão sensível a pequenas diferenças: o MAE pode não ser tão sensível a pequenas diferenças nas previsões, o que significa que modelos com pequenas melhorias podem não mostrar diferenças significativas no valor de MAE.

## 2. Erro quadrático médio (MSE)

É a média das diferenças quadráticas entre as previsões e os valores reais. Ela dá mais peso a erros grandes, sendo assim, é mais sensível a outliers do que o MAE.

**FÓRMULA:**

$$
\LARGE MSE = \frac{1}{n}\sum_{i=1}^{n}(y_i-\hat{y_i})^2
$$

Onde:

* n é o número de amostras de teste;

* yi é o valor real correspondente à amostra i;

* ŷi é a previsão do modelo para a amostra i.

Quanto menor o valor do MSE, melhor é o desempenho do modelo. Isso ocorre porque o MSE penaliza erros grandes mais do que erros pequenos, devido ao fato de elevar as diferenças ao quadrado antes de calcular a média.

<center><img src="https://drive.google.com/uc?export=view&id=1RXRi0qtNmXtkXfZlTkZqhtEeP6uz0Hx6" width="50%"></p></center>

**VANTAGENS**

* Sensibilidade a valores extremos: a MSE é sensível a valores extremos, o que significa que o modelo irá penalizar previsões que estão muito longe dos valores reais. Isso pode ser uma vantagem em situações em que é importante evitar erros grandes e previsões ruins.

* Facilidade de interpretação: a MSE é uma medida fácil de ser interpretada, pois representa a média dos erros ao quadrado.

* Utilidade em otimização de modelos: a MSE pode ser usada como função objetivo em algoritmos de otimização para encontrar os melhores valores dos parâmetros do modelo.

**DESVANTAGENS**

* Pode ser influenciada por outliers: se houver muitos outliers nos dados, a MSE pode ser influenciada de forma significativa, o que pode levar a um modelo subestimado ou superestimado.

* Não é uma medida robusta: a MSE não é uma medida robusta, o que significa que ela pode ser influenciada por pequenas variações nos dados de entrada, especialmente em conjuntos de dados pequenos.

* Não leva em conta a escala dos dados: a MSE não leva em conta a escala dos dados, o que pode levar a problemas ao comparar modelos que trabalham com diferentes unidades de medida. Nesse caso, outras métricas como o Coeficiente de Determinação (R²) ou o Mean Absolute Error (MAE) podem ser mais apropriadas.

3. Raiz quadrada do erro quadrático médio (RMSE): é a raiz quadrada do MSE. Ela também dá mais peso a erros grandes, mas sua interpretação é mais fácil de entender, pois ela é expressa nas mesmas unidades da variável dependente.

4. Coeficiente de determinação (R²): é uma medida que indica a proporção da variância da variável dependente que é explicada pelo modelo. Ele varia de 0 a 1, sendo que valores mais próximos de 1 indicam que o modelo é capaz de explicar uma grande parte da variabilidade dos dados.

5. Erro percentual absoluto médio (MAPE): é a média das diferenças percentuais absolutas entre as previsões e os valores reais. É uma medida comum em previsões financeiras, e é útil para avaliar a precisão do modelo em termos percentuais.

# Referências

* https://garba.org/posts/2022/scoring_regression/#regression-evaluation-metrics